# Assignment 2: recommender System

## Code from the lecture

In [2]:
#read data to DataFrames
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('../../DATA/movielens100k/u.user', sep='|', names=u_cols, encoding = "ISO-8859-1")

r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('../../DATA/movielens100k/u.data', sep='\t', names=r_cols, encoding = "ISO-8859-1")

m_cols=['movie_id', 'title', 'release date', 'video release date', 'IMDb_URL', 'unknown', 'Action', 'Adventure', 'Animation', 'Childrens', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
movies = pd.read_csv('../../DATA/movielens100k/u.item', sep='|', names=m_cols ,encoding = "ISO-8859-1" )

In [3]:
#gerate matrix (this can be done more efficiently!)
R=np.zeros((np.shape(users)[0],np.shape(movies)[0]))
for i in range(np.shape(ratings)[0]):
    R[ratings['user_id'][i]-1, ratings['movie_id'][i]-1]=ratings['rating'][i]

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [6]:
def CosineDist(a,b):
    return a.dot(b)/(np.linalg.norm(a)*np.linalg.norm(b))

In [7]:
#again, implementation not efficient: better use lamda opperators 
D=np.zeros((np.shape(movies)[0],np.shape(movies)[0]))
for i in range(0,np.shape(movies)[0]):
    for j in range(0,np.shape(movies)[0]):
        if i!=j:
            D[i,j]=CosineDist(R[:,i],R[:,j])

In [41]:
def getTopN(movie_id,D,N=5):
    return D[movie_id,:].argsort()[-N:]

def getIDbyName(name):
    if np.size(movies.movie_id[movies.title.str.contains(name)]) > 0:
        m = int(movies.movie_id[movies.title.str.contains(name)][:1]), str(movies.title[movies.title.str.contains(name)][:1])
        return m[0]-1
    else:
        return -1
    
def getNameByID(IDs):
    res=movies.iloc[IDs]
    return res.title

def CII(title,D):
    if getIDbyName(title) > 0:
        print ("recommending movies for: '" + str(getNameByID(getIDbyName(title)))+"'")
        return getNameByID(getTopN(getIDbyName(title),D))[::-1]
    else:
        print ("no movie title containing " + str(title) + "found...")
        
def Score_byID(ID, D, Test):
    #print ("Hit Scores for: ", getNameByID(ID))
    res_id = getTopN(ID,D)[::-1]
    print(res_id)
    res_title = getNameByID(getTopN(ID,D))[::-1]
    res_score = Test[res_id]
    return res_id, res_title, res_score, np.mean(res_score)
    
#compute scores for all test users
def test_Score(D_train, R_test):
    userScores=[]
    for i in range(R_test.shape[0]):
        userScore=0
        userTop = np.argsort(R_test[i])[-5:]
        for e in userTop:
            res_id, res_title, res_score, av_score = Score_byID(e,D_train, R_test[i])
            userScore+=av_score
        userScores.append(userScore/(5))
    return np.mean(userScores)

In [42]:
#split in train and test data
from sklearn.model_selection import train_test_split
R_train,R_test= train_test_split(R,test_size=0.1)

In [11]:
#train but only with training data
#again, implementation not efficient: better use lamda opperators 
D_train=np.zeros((np.shape(movies)[0],np.shape(movies)[0]))
for i in range(0,np.shape(movies)[0]):
    for j in range(0,np.shape(movies)[0]):
        if i!=j:
            D_train[i,j]=CosineDist(R[:,i],R[:,j])

In [43]:
res_Cos_dist=test_Score(D_train,R_test)
print("The result of the Cosine distance is",res_Cos_dist)

[224 472 150 754   0]
[ 49 180 120 116 404]
[282  13  99 236  49]
[ 14 120 281 404 124]
[698 495 131 730 418]
[271 257 314 750 299]
[116 120 404 741   6]
[332 327 312 299 244]
[120 404 116 545   0]
[ 981  830  596 1033  362]
[1488  980 1259 1487 1614]
[455 762 410 121 409]
[  37  774  793 1216  778]
[327 257 293 747 312]
[180 173 171   0 126]
[ 68 422 195  96 131]
[210 473 198 520 192]
[ 68  27 173 203 171]
[180  49 120   0 221]
[497 210 510 519 193]
[895 343 346 304 902]
[271 257 314 750 299]
[315 271 312 346 301]
[314 271 346 312 750]
[386 212 735  82  69]
[ 246 1174 1391  880  989]
[299 327 321 293 747]
[1315 1501  358  573 1264]
[299 327 681 287 332]
[ 265  262 1611 1292 1616]
[ 68 422 173 203  81]
[203 167  55 173 208]
[173  68 171  78 209]
[198 204 210 525 479]
[ 68 173  97  55 317]
[422  27 173  96 203]
[236 116 741 470 120]
[130 489 493 484 498]
[497 210 510 519 193]
[  6  49 126  55  97]
[ 99   6 507 275 149]
[ 99   8 236 126   6]
[282  13  99 236  49]
[ 49 171 173 120   0]
[1

## 4.1 Alternative Distance Measures
Implement a different distance measure, e.g. eucledian and compare the ressults.

In [19]:
#eucledian implementation
def EuclDist(a,b):
    return np.linalg.norm(a-b)

In [23]:
D_train_eucl=np.zeros((np.shape(movies)[0],np.shape(movies)[0]))
for i in range(0,np.shape(movies)[0]):
    for j in range(0,np.shape(movies)[0]):
        if i!=j:
            D_train_eucl[i,j]=EuclDist(R[:,i],R[:,j])

In [26]:
res_Eucl_dist=test_Score(D_train_eucl,R_test)
print("The result of the eucledian distance is",res_Eucl_dist)

The result of the eucledian distance is 1.5936842105263163


In [27]:
#Due to the fact, that the score equals the mean number of stars given by the users to the proposed movies by the algorithm, a higher Score is better. 
#This kind of scoring implies that the Cosine distance is better than the eucledian distance.


## 4.2 Baseline Results
Implement and compare results of
* random recommendations
* allways recommending the top 5 movies (over all users)

In [70]:
#implement and get the score of random recommendations
#the same test dataset is used as before
def get5randmovieIDs():
    nested_list=(np.round_(np.random.rand(5,1)*(np.shape(movies)[0]-1))).tolist()
    res_id =[]
    for sublist in nested_list:
        for item in sublist:
            res_id.append(int(item))
    return res_id

def Score_byID_rand(ID, Test):
    #print ("Hit Scores for: ", getNameByID(ID))
    res_id = get5randmovieIDs()
    res_title = getNameByID(getTopN(ID,D))[::-1]
    res_score = Test[res_id]
    return res_id, res_title, res_score, np.mean(res_score)
    
#compute scores for all test users
def test_Score_rand(R_test):
    userScores=[]
    for i in range(R_test.shape[0]):
        userScore=0
        userTop = np.argsort(R_test[i])[-5:]
        for e in userTop:
            res_id, res_title, res_score, av_score = Score_byID_rand(e, R_test[i])
            userScore+=av_score
        userScores.append(userScore/(5))
    return np.mean(userScores)

In [71]:
res_Cos_dist_rand=test_Score_rand(R_test)
print("The result of the Cosine distance is",res_Cos_dist_rand)

The result of the Cosine distance is 0.20294736842105265


Due to the randomness of the proposed movies the resulting score is varying, but is still much lower than the results before.

## 4.3 SVD
Implement the SVD approach. Hints: 
* use numpy.linalg.svd()
* apply SVD on $D$
* plot the singular values and decide where to cut off the reconstruction
* reconstruct and evaluate $D'$
